# Imports and Install

In [ ]:
!pip install tensorflow_model_optimization --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 4.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from keras.utils import to_categorical
from keras.models import load_model
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, Dense
import tensorflow as tf
import pathlib
from PIL import Image

# Quantized Model Architecture

In [ ]:
model_pre = tf.keras.applications.InceptionV3(
    include_top=False,
    input_shape=(299, 299, 3),
    pooling='avg',
    weights='imagenet'
)

for layer in model_pre.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
inputs = model_pre.input
outputs = model_pre.output
concatenated_model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
def apply_quantization_to_dense(layer):
        if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense): #only annotate conv. or dense layers
            return tfmot.quantization.keras.quantize_annotate_layer(layer)
        return layer

annotated_model = tf.keras.models.clone_model(
    concatenated_model,
    clone_function=apply_quantization_to_dense,
)

annotated_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 quantize_annotate (Quantiz  (None, 149, 149, 32)         864       ['input_1[0][0]']             
 eAnnotate)                                                                                       
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['quantize_annotate[0][0]']   
 Normalization)                                                                                   
                                                                                              

In [ ]:
num_classes = 94

input_shape = (1, 5, 299, 299, 3)
# no annotations for lstm
model = Sequential([
    TimeDistributed(annotated_model, batch_input_shape=input_shape),
    (LSTM(units=300, return_sequences=True)),
    TimeDistributed((Dense(units=num_classes, activation='softmax')))
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
X = np.load(r'/content/drive/My Drive/Model Files/Full_No6_NP_X_train.npy')
y = np.load(r'/content/drive/My Drive/Model Files/Full_No6_NP_y_train.npy')

In [ ]:
num_classes = 94
y_one_hot = np.zeros((y.shape[0], y.shape[1], num_classes), dtype=np.int32)
for i in range(y.shape[0]):
    y_one_hot[i] = to_categorical(y[i], num_classes=num_classes)

In [ ]:
with tfmot.quantization.keras.quantize_scope():
    model = tf.keras.models.load_model("/content/drive/My Drive/Model Files/combinedQuantAware.h5")

# Train Model

In [ ]:
# Train the model
batch_size = 32
epochs = 50
model.fit(X, y_one_hot, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/50
33/33 [==============================] - 124s 2s/step - loss: 0.6585 - accuracy: 0.8077 - val_loss: 3.3522 - val_accuracy: 0.2766
Epoch 2/50
33/33 [==============================] - 23s 703ms/step - loss: 0.6551 - accuracy: 0.8013 - val_loss: 3.1239 - val_accuracy: 0.3073
Epoch 3/50
33/33 [==============================] - 24s 720ms/step - loss: 0.2949 - accuracy: 0.9163 - val_loss: 3.4377 - val_accuracy: 0.3203
Epoch 4/50
33/33 [==============================] - 23s 709ms/step - loss: 0.1784 - accuracy: 0.9613 - val_loss: 3.1707 - val_accuracy: 0.3234
Epoch 5/50
33/33 [==============================] - 23s 706ms/step - loss: 0.1079 - accuracy: 0.9803 - val_loss: 3.0242 - val_accuracy: 0.3540
Epoch 6/50
33/33 [==============================] - 23s 712ms/step - loss: 0.0650 - accuracy: 0.9912 - val_loss: 3.1009 - val_accuracy: 0.3494
Epoch 7/50
33/33 [==============================] - 23s 713ms/step - loss: 0.0418 - accuracy: 0.9971 - val_loss: 3.1028 - val_accuracy: 0.3579
E

In [ ]:
model.save(r'/content/drive/My Drive/Model Files/combinedQuantAware.h5')

/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Quantize Aware and Convert to TFlite

In [ ]:
with tfmot.quantization.keras.quantize_scope():
    model = tf.keras.models.load_model("/content/drive/My Drive/Model Files/combinedQuantAware.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

tflite_model_file = pathlib.Path("/content/drive/My Drive/Model Files/combinedQuantAware.tflite")
tflite_model_file.write_bytes(quantized_tflite_model)

24954568